In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
from google.colab import drive
drive.mount('/content/drive')
import cv2 as cv

Mounted at /content/drive


In [ ]:
zip_filepath='/content/drive/MyDrive/graduation model/lemon_dataset.zip'

# Use the imported zipfile module here
with ZipFile(zip_filepath,'r') as zip_ref:
  zip_ref.extractall('/content/dataset')

dataset_folder='/content/dataset/lemon_quality'


In [ ]:
blemon="/content/dataset/lemon_dataset/bad_quality"
glemon="/content/dataset/lemon_dataset/good_quality"

In [ ]:
Allsizes=[]
dataset_folder='/content/dataset/lemon_dataset'

import os
for folder_name in os.listdir(dataset_folder):
  folder_path=os.path.join(dataset_folder,folder_name)
  if os.path.isdir(folder_path):
    for img in os.listdir(folder_path):
      img_path=os.path.join(folder_path,img)
      if img.endswith(('.jpg','.png','.jpeg')):
        img=cv.imread(img_path)
        img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
        if img is not None:
          Allsizes.append(img.shape)


#get images sizes
unique_sizes=set(Allsizes)
print(unique_sizes)

{(300, 300, 3), (300, 296, 3), (292, 300, 3)}


In [ ]:
images = []
labels = []
# so i will change all the images to the size 300*300
for folder_name in os.listdir(dataset_folder):
  folder_path=os.path.join(dataset_folder,folder_name)
  if os.path.isdir(folder_path):
    for img in os.listdir(folder_path):
      img_path=os.path.join(folder_path,img)
      if img.endswith(('.jpg','.png','.jpeg')):
        img=cv.imread(img_path)
        img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
        if img is not None:
          resized_img=cv.resize(img,(300,300))
          cv.imwrite(img_path,resized_img)
          images.append(resized_img)
          labels.append(folder_name)

print(f"Number of images: {len(images)}")
print(f"Number of labels: {len(labels)}")

Number of images: 2076
Number of labels: 2076


In [ ]:
images=np.array(images)
images=images/255.0


from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
labels=label_encoder.fit_transform(labels)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(images,labels,test_size=0.2,random_state=32)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(300, 300, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1028, activation='relu')(x)
x = Dense(1028, activation='relu')(x)

predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


# Save the model in HDF5 format
model.save('/content/model.hdf5', save_format='hdf5')

Epoch 1/5
52/52 [==============================] - 20s 353ms/step - loss: 0.2584 - accuracy: 0.8807 - val_loss: 0.0488 - val_accuracy: 0.9880
Epoch 2/5
52/52 [==============================] - 17s 327ms/step - loss: 0.0645 - accuracy: 0.9813 - val_loss: 0.1216 - val_accuracy: 0.9447
Epoch 3/5
52/52 [==============================] - 17s 329ms/step - loss: 0.0429 - accuracy: 0.9807 - val_loss: 0.0902 - val_accuracy: 0.9591
Epoch 4/5
52/52 [==============================] - 18s 339ms/step - loss: 0.0317 - accuracy: 0.9892 - val_loss: 0.0391 - val_accuracy: 0.9856
Epoch 5/5
52/52 [==============================] - 18s 357ms/step - loss: 0.0064 - accuracy: 0.9994 - val_loss: 0.0982 - val_accuracy: 0.9736
